In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sympy as sp
from math import factorial
from tqdm import tqdm
import csv
from scipy.sparse import csr_matrix
from scipy.sparse import identity
from scipy.sparse.linalg import spsolve
from scipy.sparse.linalg import inv

In [3]:
AREA = 2
NUMBER = 2
S = int(factorial(AREA+2+AREA**2+NUMBER)/factorial(AREA+2+AREA**2)/factorial(NUMBER))
S

45

In [4]:
# generate state dictionary direrctly
# rather than list first than dictionary
# A = 2
# N = 2
# s = [N1, N2, BP, RC, DP, R00, R10, R01, R11]
state = {}
index = [0]

def temp(x, i, index):
    x.append(i)
    generate_R(x.copy(), index)
    
def generate_R(s, index):
    if len(s)==AREA+3+AREA**2 and sum(s)==NUMBER:
        state[tuple(s)] = index[0]
        index[0] += 1
    elif len(s)>AREA+3+AREA**2 or sum(s)>NUMBER:
        return 0
    else:
        for i in range(NUMBER+1):
            temp(s.copy(), i, index)
            

for i in range(NUMBER+1):
    generate_R([i], index)
n_state = len(state)
n_queue = AREA+3+AREA**2
n_queue, n_state

(9, 45)

In [20]:
state

{(0, 0, 0, 0, 0, 0, 0, 0, 2): 0,
 (0, 0, 0, 0, 0, 0, 0, 1, 1): 1,
 (0, 0, 0, 0, 0, 0, 0, 2, 0): 2,
 (0, 0, 0, 0, 0, 0, 1, 0, 1): 3,
 (0, 0, 0, 0, 0, 0, 1, 1, 0): 4,
 (0, 0, 0, 0, 0, 0, 2, 0, 0): 5,
 (0, 0, 0, 0, 0, 1, 0, 0, 1): 6,
 (0, 0, 0, 0, 0, 1, 0, 1, 0): 7,
 (0, 0, 0, 0, 0, 1, 1, 0, 0): 8,
 (0, 0, 0, 0, 0, 2, 0, 0, 0): 9,
 (0, 0, 0, 0, 1, 0, 0, 0, 1): 10,
 (0, 0, 0, 0, 1, 0, 0, 1, 0): 11,
 (0, 0, 0, 0, 1, 0, 1, 0, 0): 12,
 (0, 0, 0, 0, 1, 1, 0, 0, 0): 13,
 (0, 0, 0, 0, 2, 0, 0, 0, 0): 14,
 (0, 0, 0, 1, 0, 0, 0, 0, 1): 15,
 (0, 0, 0, 1, 0, 0, 0, 1, 0): 16,
 (0, 0, 0, 1, 0, 0, 1, 0, 0): 17,
 (0, 0, 0, 1, 0, 1, 0, 0, 0): 18,
 (0, 0, 0, 1, 1, 0, 0, 0, 0): 19,
 (0, 0, 0, 2, 0, 0, 0, 0, 0): 20,
 (0, 0, 1, 0, 0, 0, 0, 0, 1): 21,
 (0, 0, 1, 0, 0, 0, 0, 1, 0): 22,
 (0, 0, 1, 0, 0, 0, 1, 0, 0): 23,
 (0, 0, 1, 0, 0, 1, 0, 0, 0): 24,
 (0, 0, 1, 0, 1, 0, 0, 0, 0): 25,
 (0, 0, 1, 1, 0, 0, 0, 0, 0): 26,
 (0, 0, 2, 0, 0, 0, 0, 0, 0): 27,
 (0, 1, 0, 0, 0, 0, 0, 0, 1): 28,
 (0, 1, 0, 0, 0, 0, 0, 1

In [22]:
#pij
PIJ = np.array([[0.3, 0.7],
       [0.7, 0.3]])
BROKEN = 0.3

#rate
#central
# ['lambda0','lambda1','car_move','gamma','phi','delta','r00','r01','r10','r11','broken','period']
RATE = {
    'l0':10.0,
    'l1':10.0, 
    'gamma':100.0,
    'phi':100.0,
    'n_fix' : 100,
    'delta':100.0,
    }
BCAP = 2
DCAP = 2
RIDE = np.ones((2,2))

In [49]:
P[0,]*5

array([1.5, 3.5])

In [27]:
a = np.array([[1,2],[2,3]])
a[0].dot((2,3))

8

In [57]:
a = {(1,):2, (2,):3}
# for _ in a:
#     print(list(_))
# def test(t):
#     return(t)
# list(map(test, a))
for j,i in enumerate(range(4)):
    i+=1
#[print(i) for i in [2*j for j in range(4)]]

In [34]:
S = []
for _ in state:
    S.append(list(_))
S[:5]

[[0, 0, 0, 0, 0, 0, 0, 0, 4],
 [0, 0, 0, 0, 0, 0, 0, 1, 3],
 [0, 0, 0, 0, 0, 0, 0, 2, 2],
 [0, 0, 0, 0, 0, 0, 0, 3, 1],
 [0, 0, 0, 0, 0, 0, 0, 4, 0]]

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' 

In [23]:
# 使用scipy.sparse.csr_sparse
# generate R matrix
# 换个视角，先确定哪些队列不为0，只有这些不为0的队列会使系统的状态发生改变
# s = [N1, N2, BP, RC, DP, R00, R10, R01, R11]
def N(s):
    d, c = [],[] # add one by one
    for i,n in enumerate(s[:AREA]):
        # only focus on those <> 0
        if n == NUMBER:
            #in
            for j in range(AREA):
                ls = list(s)
                ls[i], ls[AREA+j] = NUMBER-1, 1
                _c = state[tuple(s)]
                if _c not in c:
                    d.append(RIDE[j,i])
                    c.append(_c)
                else: break
            #out
            d.append(-RATE['l'+str(i)])
            c.append(state[s])
            break
        elif n == 0:
            #in
            for j in range(AREA):
                ls = list(s)
                if s[AREA+AREA*i+j] != 0:
                    ls[i], ls[AREA+AREA*i+j] = 1, ls[j]-1
                else: continue
                _c = state[tuple(s)]
                if _c not in c:
                    d.append(RATE['l'+str(i)]*PIJ[i,j])
                    c.append(_c)
        else:
            #in
            for j in s[AREA+AREA*i:2*AREA+AREA*i]:
                ls = list(s)
                if j != 0:
                    ls[i], ls[AREA+j] = ls[i]+1, ls[j]-1
                else: continue                    
                _c = state[tuple(s)]
                if _c not in c:
                    d.append(RATE['l'+str(i)]*PIJ[i,j])
                    c.append(_c)
            #out
            d.append(-RATE['l'+str(i)])
            c.append(state[s])
    #print(d,c)
    print('n',s,d)
    return c,d

def R(s):
    d, c = [],[] # add one by one
    for i in range(AREA):
        start = i*AREA+AREA
        for j,n in enumerate(s[start:start+AREA]):
            if n == NUMBER:
                #in
                ls = list(s)
                ls[start+j], ls[i] = NUMBER-1, 1
                _c = state[tuple(s)]
                if _c not in c:
                    d.append(RATE['l'+str(i)]*PIJ[i,j])
                    c.append(_c)
                else: break
                #out, back to work or broken
                ls = list(s)
                ls[start+j], ls[i] = NUMBER-1, 1
                d.append(-RIDE[i,j]*s[start+j])
                c.append(state[s])
                break
            else:
                #in, back to work or broken
                if s[i]!=0:
                    ls = list(s)
                    ls[i], ls[start+j] = ls[i]-1, ls[start]+1
                    _c = state[tuple(s)]
                    if _c not in c:
                        d.append(RIDE[i,j]*s[start+j]*(1-BROKEN))
                        c.append(_c)
                if s[-3]!=0:
                    ls = list(s)
                    ls[-3], ls[start+j] = ls[-3]-1, ls[start+j]+1
                    _c = state[tuple(s)]
                    if _c not in c:
                        d.append(RIDE[i,j]*s[start+j]*BROKEN)
                        c.append(_c)
                #out
                if n!=0:
                    d.append(-RIDE[i,j]*s[start+j])
                    c.append(state[s])
    print('r',s,d)
    return c,d

def BP(s):
    d, c = [],[] # add one by one
    if s[-3]==BCAP:
        # rate out
        ls = list(s)
        ls[-3], ls[-2] = ls[-3]-BCAP, ls[-2]+BCAP
        _c = state[tuple(s)]
        if _c not in c:
            d.append(-RATE['gamma'])
            c.append(_c)
    elif s[-3]==0:
        # rate in 
        if s[-2]>=BCAP:
            ls = list(s)
            ls[-3], ls[-2] = ls[-3]+BCAP, ls[-2]-BCAP
            _c = state[tuple(s)]
            if _c not in c:
                d.append(RATE['gamma'])
                c.append(_c)
    else:
        for i in range(AREA):
            start = i*AREA+AREA
            for j,n in enumerate(s[start:start+AREA]):
                ls = list(s)
                ls[start+j], ls[-3] = ls[start+j]+1, ls[-3]-1
                _c = state[tuple(s)]
                if _c not in c:
                    d.append(RIDE[i,j]*s[start+j]*(BROKEN))
                    c.append(_c)
    print('bp',s,d)
    return c,d

def RC(s):
    d, c = [],[] # add one by one
    # in already considered in the previous part
    # out 
    if s[-2]>0 and s[-1]<DCAP:
        ls = list(s)
        ls[-1], ls[-2] = ls[-1]+1, ls[-2]-1
        _c = state[tuple(s)]
        if _c not in c:
            d.append(-RATE['phi']*min([RATE['n_fix'], s[-2]]))
            c.append(_c)
    print('rc',s,d)
    return c,d
    

def DP(s):
    d, c = [],[] # add one by one
    # in has been considered in the previous part 
    # out
    if s[-1] == DCAP:
        ls = list(s)
        ls[-1]-=DCAP
        for i in range(AREA):
            ls[i]+=DCAP/AREA
        _c = state[tuple(s)]
        if _c not in c:
            d.append(-RATE['delta'])
            c.append(_c)
    print('dp',s,d)
    return c,d

def write_matrix(f,s):
    re1, re2 = f
    #print(re1,re2)
    if re1:
        l = len(re1)
        #print(l)
        for i in range(l):
            Row.append(state[s]) 
            Col.append(re1[i])
            Dat.append(re2[i])
            #print(Row,Col,Dat)

def generate_equations(s):


    '''deal with N'''
    write_matrix(N(s),s)
    '''deal with R'''
    write_matrix(R(s),s)
    '''deal with BP'''
    write_matrix(BP(s),s)
    '''deal with RC'''
    write_matrix(RC(s),s)
    '''deal with DP'''
    write_matrix(DP(s),s)
    

Row, Col, Dat = [], [], []
list(map(generate_equations, state))
    
    
    

n (0, 0, 0, 0, 0, 0, 0, 0, 2) []
r (0, 0, 0, 0, 0, 0, 0, 0, 2) []
bp (0, 0, 0, 0, 0, 0, 0, 0, 2) []
rc (0, 0, 0, 0, 0, 0, 0, 0, 2) []
dp (0, 0, 0, 0, 0, 0, 0, 0, 2) [-100.0]
n (0, 0, 0, 0, 0, 0, 0, 1, 1) []
r (0, 0, 0, 0, 0, 0, 0, 1, 1) []
bp (0, 0, 0, 0, 0, 0, 0, 1, 1) []
rc (0, 0, 0, 0, 0, 0, 0, 1, 1) [-100.0]
dp (0, 0, 0, 0, 0, 0, 0, 1, 1) []
n (0, 0, 0, 0, 0, 0, 0, 2, 0) []
r (0, 0, 0, 0, 0, 0, 0, 2, 0) []
bp (0, 0, 0, 0, 0, 0, 0, 2, 0) [100.0]
rc (0, 0, 0, 0, 0, 0, 0, 2, 0) [-200.0]
dp (0, 0, 0, 0, 0, 0, 0, 2, 0) []
n (0, 0, 0, 0, 0, 0, 1, 0, 1) []
r (0, 0, 0, 0, 0, 0, 1, 0, 1) [0.0]
bp (0, 0, 0, 0, 0, 0, 1, 0, 1) [0.0]
rc (0, 0, 0, 0, 0, 0, 1, 0, 1) []
dp (0, 0, 0, 0, 0, 0, 1, 0, 1) []
n (0, 0, 0, 0, 0, 0, 1, 1, 0) []
r (0, 0, 0, 0, 0, 0, 1, 1, 0) [0.0]
bp (0, 0, 0, 0, 0, 0, 1, 1, 0) [0.0]
rc (0, 0, 0, 0, 0, 0, 1, 1, 0) [-100.0]
dp (0, 0, 0, 0, 0, 0, 1, 1, 0) []
n (0, 0, 0, 0, 0, 0, 2, 0, 0) []
r (0, 0, 0, 0, 0, 0, 2, 0, 0) [0.0]
bp (0, 0, 0, 0, 0, 0, 2, 0, 0) [-100.0]
rc (0, 0, 

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [120]:
Row,Col,Dat

([1,
  2,
  3,
  3,
  4,
  4,
  4,
  5,
  5,
  6,
  6,
  7,
  7,
  7,
  8,
  8,
  8,
  8,
  9,
  9,
  9,
  10,
  10,
  11,
  11,
  11,
  12,
  12,
  12,
  12,
  13,
  13,
  13,
  14,
  14,
  14,
  15,
  15,
  16,
  16,
  16,
  17,
  17,
  17,
  17,
  18,
  18,
  18,
  19,
  19,
  19,
  20,
  20,
  20,
  21,
  21,
  22,
  22,
  22,
  23,
  23,
  23,
  23,
  24,
  24,
  24,
  25,
  25,
  25,
  26,
  26,
  26,
  27,
  27,
  27,
  28,
  28,
  29,
  29,
  29,
  30,
  30,
  30,
  31,
  31,
  31,
  31,
  32,
  32,
  32,
  32,
  33,
  33,
  33,
  34,
  34,
  34,
  35,
  35,
  35,
  36,
  36,
  37,
  37,
  37,
  38,
  38,
  38,
  39,
  39,
  39,
  40,
  40,
  40,
  41,
  41,
  41,
  41,
  42,
  42,
  42,
  42,
  43,
  43,
  43,
  44,
  44,
  44],
 [1,
  2,
  3,
  3,
  4,
  4,
  4,
  5,
  5,
  6,
  6,
  7,
  7,
  7,
  8,
  8,
  8,
  8,
  9,
  9,
  9,
  10,
  10,
  11,
  11,
  11,
  12,
  12,
  12,
  12,
  13,
  13,
  13,
  14,
  14,
  14,
  15,
  15,
  16,
  16,
  16,
  17,
  17,
  17,
  17,
  1

In [24]:
A = csr_matrix((Dat, (Row, Col)), dtype=np.float)

In [27]:
list(A.toarray())

[array([-100.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.]),
 array([   0., -100.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.]),
 array([   0.,    0., -100.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
  

In [26]:
spsolve(A, np.zeros((len(state))))

/Users/valarian/python/miniconda3/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:193: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)


array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan])